In [3]:
import torch.nn as nn

import sys
sys.path.append("..")

import torch

import math

from mamba_ssm import Mamba

from mambaCText import MambaCText

from muse_maskgit_pytorch import LayerNorm

from mamba_ssm.ops.triton.layernorm import layer_norm_fn
from einops import repeat

from masked_model import MaskedModel, SequenceModelWrapper

In [4]:
from muse_maskgit_pytorch import Attention

In [3]:
def cosine_schedule(t):
    return torch.cos(t * math.pi * 0.5)

In [22]:
class MambaIT_CText(nn.Module):
  def __init__(
      self,
      token_size,
      depth,
      d_state = 16,
      d_conv = 4,
      expand = 2,
  ):
    super().__init__()
    self.token_size = token_size
    self.mamba_layers = nn.ModuleList([MambaCText(d_model = token_size, d_state = d_state, d_conv = d_conv, expand = expand) for _ in range(depth)])
    
    self.attention = Attention(token_size, cross_attend = True)
    
    self.norm = LayerNorm(token_size)

  def forward(
      self,
      x,
      context,
      context_mask
  ):

    seq_len = x.shape[1]

    context_mask = repeat(context_mask, 'b t -> b t s', s = self.token_size)
    context = torch.where(context_mask, context, torch.zeros_like(context))

    context = self.attention(x, context)

    print(context.shape)

    for mamba_layer in self.mamba_layers:
      x = mamba_layer(x, context)
      x = self.norm(x)

    return x